# Urban Growth Boundary (UGB)

In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#import API keys: 
from config import api_key

#save data in a CSV
output_file = "output_data/ugb.csv"


In [ ]:
# Start of code for Question 1: housing types

In [ ]:
# End of code for Question 1: housing types

In [ ]:
# Start of code for Question 2: public transportation

In [ ]:
# End of code for Question 2: public transportation

In [ ]:
# Start of code for Question 3: population growth

In [ ]:
# End of code for Question 3: population growth

In [ ]:
# Start of code for Question 4: population density

In [ ]:
# End of code for Question 4: population density